In [ ]:
pip install ucimlrepo

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd
import tensorflow as tf
from ucimlrepo import fetch_ucirepo

# Fetching the dataset
student_performance = fetch_ucirepo(id=320)

# Extracting features and targets
X = student_performance.data.features
y = student_performance.data.targets

# Converting features and targets to Pandas DataFrames
X_df = pd.DataFrame(X, columns=student_performance.metadata.features)
y_df = pd.DataFrame(y, columns=student_performance.metadata.targets)

# Concatenating features and targets into a single DataFrame
student_df = pd.concat([X_df, y_df], axis=1)

In [ ]:
# Creating pass/fail category for 'G3'
# True ('pass' or 1) if >= 10, else False ('fail' or 0)
student_df["G3"] = student_df["G3"].astype(int)
student_df["pass_fail"] = (student_df["G3"] >= 10).astype(int)

In [ ]:
# Dropping original 'G3'
student_df = student_df.drop(columns = "G3")

# Encoding

In [ ]:
# Converting categorical data to numeric with 'pd.get_dummies'
student_df = pd.get_dummies(student_df)

In [ ]:
# Gathering all column names
student_df_column_names_list = student_df.columns.tolist()
print(student_df_column_names_list)

['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2', 'pass_fail', 'school_GP', 'school_MS', 'sex_F', 'sex_M', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'schoolsup_no', 'schoolsup_yes', 'famsup_no', 'famsup_yes', 'paid_no', 'paid_yes', 'activities_no', 'activities_yes', 'nursery_no', 'nursery_yes', 'higher_no', 'higher_yes', 'internet_no', 'internet_yes', 'romantic_no', 'romantic_yes']


In [ ]:
# Removing column names (those that are 'to keep' based on Model Inspection results)
columns_to_keep = ['pass_fail', 'G2', 'G1', 'failures', 'internet_yes', 'absences', 'paid_yes', 'address_R', 'nursery_no', 'Walc', 'goout']
columns_to_drop = list(set(student_df_column_names_list) - set(columns_to_keep))
student_df = student_df.drop(columns = columns_to_drop)
print(student_df.columns.tolist())

['failures', 'goout', 'Walc', 'absences', 'G1', 'G2', 'pass_fail', 'address_R', 'paid_yes', 'nursery_no', 'internet_yes']


In [ ]:
# Splitting preprocessed data into features and target arrays
X = student_df.drop(columns = "pass_fail", axis = 1)
y= student_df["pass_fail"]

# Splitting the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

# Scaling

In [ ]:
# Creating a StandardScaler instance
scaler = StandardScaler()

# Fitting the StandardScaler
X_scaler = scaler.fit(X_train)

# Scaling the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Compile, Train, and Evaluate the Model

In [ ]:
# Defining the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

# Optimized hyperparameters based on Chalisa's educated trial-and-error work
features = X_train_scaled.shape[1]
nodes_layer1 = 100
nodes_layer2 = 50
nodes_layer3 = 30
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = nodes_layer1, input_dim = features, activation = "relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = nodes_layer2, activation = "relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units = nodes_layer3, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Checking the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 100)               1100      
                                                                 
 dense_5 (Dense)             (None, 50)                5050      
                                                                 
 dense_6 (Dense)             (None, 30)                1530      
                                                                 
 dense_7 (Dense)             (None, 1)                 31        
                                                                 
Total params: 7711 (30.12 KB)
Trainable params: 7711 (30.12 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compiling the model
nn.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [ ]:
# Training the model
fitted_model = nn.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100
16/16 [==============================] - 1s 2ms/step - loss: 0.6035 - accuracy: 0.7942
Epoch 2/100
16/16 [==============================] - 0s 2ms/step - loss: 0.4282 - accuracy: 0.8580
Epoch 3/100
16/16 [==============================] - 0s 2ms/step - loss: 0.2935 - accuracy: 0.8807
Epoch 4/100
16/16 [==============================] - 0s 2ms/step - loss: 0.2186 - accuracy: 0.9259
Epoch 5/100
16/16 [==============================] - 0s 2ms/step - loss: 0.1862 - accuracy: 0.9321
Epoch 6/100
16/16 [==============================] - 0s 2ms/step - loss: 0.1647 - accuracy: 0.9362
Epoch 7/100
16/16 [==============================] - 0s 2ms/step - loss: 0.1490 - accuracy: 0.9403
Epoch 8/100
16/16 [==============================] - 0s 2ms/step - loss: 0.1377 - accuracy: 0.9403
Epoch 9/100
16/16 [==============================] - 0s 2ms/step - loss: 0.1298 - accuracy: 0.9403
Epoch 10/100
16/16 [==============================] - 0s 2ms/step - loss: 0.1206 - accuracy: 0.9465
Epoch 11/

In [ ]:
# Evaluating the model using training data
model_loss, model_accuracy = nn.evaluate(X_train_scaled,y_train,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

16/16 - 0s - loss: 0.0012 - accuracy: 1.0000 - 177ms/epoch - 11ms/step
Loss: 0.0011889534071087837, Accuracy: 1.0


In [ ]:
# Evaluating the model using test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

6/6 - 0s - loss: 0.6583 - accuracy: 0.9080 - 29ms/epoch - 5ms/step
Loss: 0.6583045721054077, Accuracy: 0.907975435256958
